## 作业4

请根据这个基本的框架程序进行扩展，使用你的语料库进行训练。并完成 3 个名词各自最相近的 Top 10 个词的检索

In [1]:
%matplotlib inline
from collections import Counter
import jieba
import jieba.posseg as pseg
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)

1.0.1


### 定义参数

In [2]:
# 决定了 embedding 的维度 （隐层节点数）
word_embedding_dim = 50
# 决定了词表数量, 预留一个未登录词
vocab_size = 5000 + 1
UNK_IDX = 0

### 定义变量

In [3]:
word_embedding = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim]))
input_data = tf.placeholder(tf.int32, shape=[None, 2], name='input_data')
input_embeds = tf.nn.embedding_lookup(word_embedding, input_data)

In [34]:
input_data

<tf.Tensor 'input_data:0' shape=(?, 2) dtype=int32>

In [35]:
input_embeds

<tf.Tensor 'embedding_lookup:0' shape=(?, 2, 50) dtype=float32>

### 词向量相加

In [4]:
context_embeds = tf.reduce_sum(input_embeds, axis=1)

In [5]:
context_embeds

<tf.Tensor 'Sum:0' shape=(?, 50) dtype=float32>

### 映射到 N 个词的概率分布

In [6]:
# raw_output 是一个 vocab_size 维的数据，对比 labels 计算 cost
# 假设输入一组（也就是 两个词），输出因为词向量相加过了，所以就是一个词的词向量：one-hot？
raw_output = tf.layers.dense(context_embeds, vocab_size)
# 如果输入一组，输出的 softmax 是预测的 one-hot 的概率分布？最可能的那个输出词概率最大？
output = tf.nn.softmax(raw_output)

In [7]:
raw_output

<tf.Tensor 'dense/BiasAdd:0' shape=(?, 5001) dtype=float32>

In [8]:
output

<tf.Tensor 'Softmax:0' shape=(?, 5001) dtype=float32>

### cost

In [9]:
# 样本的 labels 也需要用 placeholder 放置
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

# 因为我们每个样本的 label 只有一个，使用稠密的 softmax 算 cost 及求导太浪费了。这里使用 sparse 版本即可。
# 如果你的 label 是完整的 N 个词上的概率分布，这时候可以使用 tf.nn.softmax_cross_entropy_with_logits
cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=raw_output, labels=labels)

### 读取语料，生成训练数据

In [60]:
line_no = 0
words = []
with open('../AssisantEvaluate/xiyouji.txt', 'r') as f:
    for line in f.readlines():
        line_no += 1
        if line_no > 10000:
            break
        word = pseg.cut(line.strip().decode('utf-8')) # 去掉末尾的 '\n'
        for w,f in word:
            if f == 'x':
                continue
            words.append(w)

In [62]:
len(words)

132268

In [63]:
# 统计词频
word_cnt = Counter(words)

In [64]:
len(word_cnt)

19387

In [65]:
# 高频截断
vocab = [i[0] for i in word_cnt.most_common(vocab_size - 1)]

In [66]:
vocab2 = [i for i in word_cnt.most_common(vocab_size - 1)]

In [67]:
# 插入未登录词
vocab.insert(UNK_IDX, 'UNK')

In [68]:
len(vocab)

5001

In [69]:
# 映射 id
word_ids = [vocab.index(word) if (word in vocab) else 0 for word in words]

In [70]:
len(word_ids)

132268

In [72]:
# 生成训练数据
inputs_train = np.asarray([[word_ids[i-1], word_ids[i+1]] for i in range(1, len(word_ids) - 1)])
labels_train = np.asarray(word_ids[1:-1])

In [73]:
inputs_train.shape

(132266, 2)

In [74]:
labels_train.shape

(132266,)

### 训练模型

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.0002).minimize(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    dummy_feed_dict = {input_data: inputs_train,
                       labels: labels_train}
    for i in range(10):
        sess.run(train_step, feed_dict=dummy_feed_dict)
        if i % 2 == 0:
            print("Iteration %d" % i)
            
            # cost 情况
            cost_array = cost.eval(feed_dict=dummy_feed_dict)
            print("Cost 矩阵：")
            print(cost_array)
            print("Cost 矩阵 shape：")
            print(cost_array.shape)
            print('---')
            
            # Output 情况
            output_array = output.eval(feed_dict=dummy_feed_dict)
            print("Output 矩阵：")
            print(output_array)
            print("Output 矩阵 shape：")
            print(output_array.shape)
            print(output_array[0].shape)
            print('---')
            
            # 查看输出中 ID == 30 的概率            
            print("Probability: %f" % output_array[0, 30])
            print("------")
            # 词向量是 context_embeds 吗？
            allwords_embedding = context_embeds.eval(feed_dict=dummy_feed_dict)
            raw_outputs = raw_output.eval(feed_dict=dummy_feed_dict)

### 计算近义词

In [51]:
def sim_words(word, top_k=10):
    
    a = allwords_embedding
    id_ = vocab.index(word.decode('utf-8'))
    
    len_array = np.diag(a.dot(a.T)) ** 0.5
    cos_array = a.dot(a[id_]) / len_array
    cos_dict = dict(zip(cos_array, words[1:-1]))
    sort_cos_dict = sorted(cos_dict.items(), key=lambda x: x[0], reverse=True)
    
    
    return sort_cos_dict[0:top_k], len(cos_array), len(cos_dict)

In [56]:
len(Counter(words[1:-1]))

4270

In [52]:
sim_words('三藏')

([(7.0793791, u'\u5927\u9053'),
  (6.6364622, u'\u4eba\u751f'),
  (6.6232367, u'\u81f3\u6b64'),
  (6.607203, u'\u82d4\u85d3'),
  (6.5855713, u'\u72ec\u89d2'),
  (6.5578933, u'\u518d'),
  (6.5525885, u'\u65b9\u6cd5'),
  (6.539609, u'\u4f1a'),
  (6.5385327, u'\u600e\u4e48'),
  (6.5354853, u'\u8d77\u4f0f')],
 12949,
 11250)

In [46]:
len(result)

11250

In [45]:
result = sim_words('三藏')
for k,v in result[0:10]:
    print k,v

7.07938 大道
6.63646 人生
6.62324 至此
6.6072 苔藓
6.58557 独角
6.55789 再
6.55259 方法
6.53961 会
6.53853 怎么
6.53549 起伏


In [32]:
for w,c in vocab2[0:30]:
    print("%s %d" % (w, c))

道 299
了 273
我 265
他 215
的 213
你 207
那 160
是 146
三藏 139
行者 117
有 115
去 95
这 92
又 92
来 90
也 84
在 83
不 76
师父 70
见 65
得 65
就 53
与 53
着 51
个 50
一 49
菩萨 46
说 45
却 45
上 43
